# Super-résolution d'image avec un CAE (Convolutional Autoencoder)

Ce notebook **Google Colab** montre une pipeline complète en Python:

1. Charger des images haute qualité (CIFAR-10).
2. Dégrader les images pour générer des entrées basse qualité.
3. Entraîner un **CAE** pour reconstruire une image de meilleure qualité.
4. Visualiser les résultats (entrée LQ vs sortie du CAE vs cible HQ).

## 1) Imports et configuration

> Sur Colab, TensorFlow est généralement déjà installé.

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

print("TensorFlow:", tf.__version__)
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

## 2) Charger les données (CIFAR-10)

Pour un notebook rapide sur Colab, CIFAR-10 est pratique (images 32x32).
L'image "haute qualité" cible sera l'image originale, et l'image basse qualité sera une version dégradée.

In [ ]:
(x_train, _), (x_test, _) = tf.keras.datasets.cifar10.load_data()

x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

print("Train:", x_train.shape, x_train.dtype)
print("Test:", x_test.shape, x_test.dtype)

## 3) Générer des images basse qualité (pipeline de dégradation)

On simule une image basse qualité via:
- réduction de résolution (32→16),
- ré-agrandissement (16→32),
- ajout d'un léger bruit gaussien.

In [ ]:
def degrade_images(images, downscale_size=16, noise_std=0.03):
    # Downsample puis upsample (perte d'information)
    low = tf.image.resize(images, [downscale_size, downscale_size], method="area")
    low = tf.image.resize(low, [32, 32], method="bicubic")

    # Bruit gaussien
    noise = tf.random.normal(tf.shape(low), mean=0.0, stddev=noise_std, dtype=low.dtype)
    low = tf.clip_by_value(low + noise, 0.0, 1.0)
    return low

x_train_lq = degrade_images(x_train)
x_test_lq = degrade_images(x_test)

print("Train LQ:", x_train_lq.shape)
print("Test LQ:", x_test_lq.shape)

## 4) Construire le modèle CAE

Le modèle encode d'abord l'image (compression), puis la décode (reconstruction).

In [ ]:
def build_cae(input_shape=(32, 32, 3)):
    inputs = tf.keras.Input(shape=input_shape)

    # Encodeur
    x = tf.keras.layers.Conv2D(64, 3, padding="same", activation="relu")(inputs)
    x = tf.keras.layers.MaxPooling2D(2)(x)  # 16x16
    x = tf.keras.layers.Conv2D(128, 3, padding="same", activation="relu")(x)
    x = tf.keras.layers.MaxPooling2D(2)(x)  # 8x8

    # Bottleneck
    x = tf.keras.layers.Conv2D(256, 3, padding="same", activation="relu")(x)

    # Décodeur
    x = tf.keras.layers.Conv2DTranspose(128, 3, strides=2, padding="same", activation="relu")(x)  # 16x16
    x = tf.keras.layers.Conv2DTranspose(64, 3, strides=2, padding="same", activation="relu")(x)   # 32x32
    outputs = tf.keras.layers.Conv2D(3, 3, padding="same", activation="sigmoid")(x)

    model = tf.keras.Model(inputs, outputs, name="cae_super_resolution")
    return model

model = build_cae()
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="mae", metrics=["mse"])
model.summary()

## 5) Entraîner le CAE

Pour une démo Colab rapide, on peut sous-échantillonner le dataset.
Tu peux augmenter `subset_size` et `epochs` pour une meilleure qualité.

In [ ]:
subset_size = 20000
x_train_lq_sub = x_train_lq[:subset_size]
x_train_hq_sub = x_train[:subset_size]

history = model.fit(
    x_train_lq_sub,
    x_train_hq_sub,
    validation_split=0.1,
    epochs=12,
    batch_size=128,
    shuffle=True,
)

## 6) Évaluer et visualiser les résultats

In [ ]:
test_loss, test_mse = model.evaluate(x_test_lq, x_test, verbose=0)
print(f"Test MAE: {test_loss:.4f} | Test MSE: {test_mse:.4f}")

In [ ]:
def show_results(low_quality, predictions, targets, n=6):
    plt.figure(figsize=(12, 6))
    for i in range(n):
        # Entrée basse qualité
        ax = plt.subplot(3, n, i + 1)
        plt.imshow(low_quality[i])
        plt.title("Entrée LQ")
        plt.axis("off")

        # Sortie CAE
        ax = plt.subplot(3, n, i + 1 + n)
        plt.imshow(predictions[i])
        plt.title("Sortie CAE")
        plt.axis("off")

        # Cible haute qualité
        ax = plt.subplot(3, n, i + 1 + 2*n)
        plt.imshow(targets[i])
        plt.title("Cible HQ")
        plt.axis("off")
    plt.tight_layout()
    plt.show()

num_show = 6
preds = model.predict(x_test_lq[:num_show])
show_results(x_test_lq[:num_show], preds, x_test[:num_show], n=num_show)

## 7) (Optionnel) Sauvegarder le modèle

In [ ]:
save_path = "cae_super_resolution.keras"
model.save(save_path)
print("Modèle sauvegardé dans:", save_path)

## 8) Idées d'amélioration

- Ajouter des **skip connections** (style U-Net).
- Utiliser une perte perceptuelle (VGG) en plus de MAE/MSE.
- Entraîner sur un dataset HD (DIV2K, Flickr2K, etc.).
- Tester différentes dégradations (flou, compression JPEG, bruit plus fort).